## Notebook to produce DP1 fields and number of images summary table, table 1

In [ ]:
# Generic python packages
import numpy as np
import pandas as pd

# LSST Science Pipelines 
from lsst.daf.butler import Butler

# Set a standard figure size to use
from lsst.utils.plotting import publication_plots, get_multiband_plot_colors

In [ ]:
# Setyp publication style
publication_plots.set_rubin_plotstyle()
colors = get_multiband_plot_colors()
bands = colors.keys()  # important to get the right order for plot legends

In [ ]:
# Set up Butller
instrument = 'LSSTComCam'
collections = ['LSSTComCam/raw/all', 
               'LSSTComCam/DP1/defaults', 
               'LSSTComCam/calib',
               'LSSTComCam/runs/DRP/DP1/v29_0_0/DM-50260',
               'skymaps', ]
skymap = 'lsst_cells_v1'
butler = Butler("/repo/dp1",
                instrument=instrument, 
                collections=collections, 
                skymap=skymap)
registry = butler.registry
skymap = butler.get('skyMap', skymap=skymap)

In [ ]:
# target fields and description. 
# This needs to be changed to extract from the data and not hardcoded
fields_table = {
    "Field": ['47_Tuc','ECDFS','EDFS_comcam','Fornax_dSph','Rubin_SV_095_-25','Rubin_SV_38_7','Seagull'],
    "Description": ['47 Tuc Globular Cluster', 'Extended Chandra Deep Field South', 
                    "Rubin Euclid Deep Field South", "Fornax Dwarf Spheroidal Galaxy",
                    "Low Galactic Latitude Field",  "Low Ecliptic Latitude Field", "Seagull Nebula"],
    "RA": [6.02, 53.13, 59.10, 40.00, 95, 37.86, 106.23],
    "DEC": [-72.08, -28.10, -48.73, -34.45, -25.00, 6.98, -10.51 ]
}

df_fields = pd.DataFrame(fields_table)

In [ ]:
# Total number of exposures in DP1
exposures = registry.queryDimensionRecords('exposure')
exposures.count()

In [ ]:
# Extract exposures data into a data frame
df = pd.DataFrame(columns=['id', 'target','type', 'reason','filter','ra', 'dec'])
for count, info in enumerate(exposures):
    try:
        df.loc[count] = [info.id, info.target_name, info.observation_type,
                         info.observation_reason, info.physical_filter, 
                         info.tracking_ra, info.tracking_dec]
    except: 
        print(">>>   Unexpected error:", sys.exc_info()[0])
assert len(df) == exposures.count()

In [ ]:
# Physical filter -> band
df['band'] = df['filter'].str.split('_').str[0]

In [ ]:
table = df.groupby(['target', 'band']).size().unstack(fill_value=0)[bands]
table

In [ ]:
latex_code = table.to_latex(index=True, header=True)
with open("../tables/dp1_visit-summary.tex", "w") as f:
    f.write(latex_code)